In [6]:
import numpy as np
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import tqdm as tqdm
from lxml import html
import time

In [7]:
def getSoup_test(link):
        r = requests.get(link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'html')

In [8]:
bf_test = getSoup_test('http://wiki.lspace.org/mediawiki/List_of_Pratchett_characters')

In [9]:
links_test = bf_test.find('span',{'id':'Unseen_University_and_the_Wizards'}).parent.previous.previous.parent.parent.parent.find_all('li')
links_test_list =  ['http://wiki.lspace.org' + li.find('a')['href'] for li in links_test]
links_test_list

['http://wiki.lspace.org/mediawiki/Mrs._Cake',
 'http://wiki.lspace.org/mediawiki/Fred_Colon',
 'http://wiki.lspace.org/mediawiki/Mrs._Marietta_Cosmopilite',
 'http://wiki.lspace.org/mediawiki/Detritus',
 'http://wiki.lspace.org/mediawiki/Cut-Me-Own-Throat_Dibbler',
 'http://wiki.lspace.org/mediawiki/Dorfl',
 'http://wiki.lspace.org/mediawiki/Rufus_Drumknott',
 'http://wiki.lspace.org/mediawiki/Gaspode',
 'http://wiki.lspace.org/mediawiki/Carrot_Ironfoundersson',
 'http://wiki.lspace.org/mediawiki/Leonard_of_Quirm',
 'http://wiki.lspace.org/mediawiki/Cheery_Littlebottom',
 'http://wiki.lspace.org/mediawiki/Nobby_Nobbs',
 'http://wiki.lspace.org/mediawiki/Lady_Sybil_Ramkin',
 'http://wiki.lspace.org/mediawiki/Foul_Ole_Ron',
 'http://wiki.lspace.org/mediawiki/Reg_Shoe',
 'http://wiki.lspace.org/mediawiki/Mr._Slant',
 'http://wiki.lspace.org/mediawiki/Angua_von_%C3%9Cberwald',
 'http://wiki.lspace.org/mediawiki/Havelock_Vetinari',
 'http://wiki.lspace.org/mediawiki/Samuel_Vimes',
 'http:/

In [10]:
bf_char_test = getSoup_test(links_test_list[1])

In [11]:
books_list_test = bf_char_test.find('div', {'id':'mw-content-text'}).find('table').find(text = 'Books').parent.parent.parent.find_next_siblings('td')

In [12]:
bf_char_test.find('div', {'id':'mw-content-text'}).find('table').find(text = 'Books').parent.parent.parent.parent.findAll('i')

[<i><a href="/mediawiki/Book:Guards!_Guards!" title="Book:Guards! Guards!">Guards! Guards!</a></i>,
 <i><a href="/mediawiki/Book:Men_at_Arms" title="Book:Men at Arms">Men at Arms</a></i>,
 <i><a href="/mediawiki/Book:Soul_Music" title="Book:Soul Music">Soul Music</a></i>,
 <i><a href="/mediawiki/Book:Feet_of_Clay" title="Book:Feet of Clay">Feet of Clay</a></i>,
 <i><a href="/mediawiki/Book:Jingo" title="Book:Jingo">Jingo</a></i>,
 <i><a href="/mediawiki/Book:The_Fifth_Elephant" title="Book:The Fifth Elephant">The Fifth Elephant</a></i>,
 <i><a href="/mediawiki/Book:The_Truth" title="Book:The Truth">The Truth</a></i>,
 <i><a href="/mediawiki/Book:The_Last_Hero" title="Book:The Last Hero">The Last Hero</a></i>,
 <i><a href="/mediawiki/Book:Night_Watch" title="Book:Night Watch">Night Watch</a></i>,
 <i><a href="/mediawiki/Book:Thud!" title="Book:Thud!">Thud!</a></i>]

In [37]:
[i.text for i in books_list_test]

[' Feet of ClayJingoThe Fifth ElephantThud!\n']

In [13]:
class BookTaker: 
    '''
    Getting infos from webpage 
    '''
    def __init__(self, link):
        '''
        Input link to search Books appered in 
        '''
        self.link = link

    def getSoup(self):
        r = requests.get(self.link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'html')
    
    def getBooks(self):
        bf = self.getSoup()
        books_list = bf.find('div', {'id':'mw-content-text'}).find('table').find(text = 'Books').parent.parent.parent.parent.findAll('i')
        return [i.text.strip()  for i in books_list]
    
    def getName(self):
        bf = self.getSoup()
        name = bf.find('div', {'id':'mw-content-text'}).find('table').find('tr').text.strip()
        return name

In [110]:
BookTaker(links_test_list[20]).getBooks()

['Men at Arms',
 'Feet of Clay',
 'Jingo',
 'The Fifth Elephant',
 'Night Watch',
 'Thud!',
 'Snuff']

In [14]:
class Downloader:
    '''
    Download all links for bookchecker to initiate
    '''
    def __init__(self, link):
        '''
        Give link to extract links of given webpages
        '''
        self.link = link
    
    def getSoup(self):
        r = requests.get(self.link)
        r.encoding = 'UTF-8'
        return BeautifulSoup(r.text, 'lxml')

    def getLinks(self):
        wp = self.getSoup()
        links = wp.find('span',{'id':'Unseen_University_and_the_Wizards'}).parent.previous.previous.parent.parent.parent.find_all('li')
        return ['http://wiki.lspace.org' + li.find('a')['href'] for li in links]

    def getDataFrame(self):
        links = self.getLinks()
        name_list = [BookTaker(i).getName() for i in links]
        books_list = [BookTaker(i).getBooks() for i in links]
        return pd.DataFrame({"Character":name_list, "Books":books_list})
        

In [15]:
CharBookDF = Downloader('http://wiki.lspace.org/mediawiki/List_of_Pratchett_characters').getDataFrame()


In [18]:
CharBookDF.head()

,Character,Books
0,Mrs Cake,"[Reaper Man, Men at Arms]"
1,Fred Colon,"[Guards! Guards!, Men at Arms, Soul Music, Fee..."
2,Mrs. Marietta Cosmopilite,[Moving Pictures]
3,Detritus,"[Guards! Guards!, Moving Pictures, Men at Arms..."
4,Cut-Me-Own-Throat Dibbler,"[Guards! Guards!, Moving Pictures, Reaper Man,..."


In [19]:
CharBookDF_ext = pd.concat([CharBookDF['Character'], CharBookDF['Books'].apply(pd.Series)], axis = 1)
CharBookDF_ext

,Character,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Mrs Cake,Reaper Man,Men at Arms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fred Colon,Guards! Guards!,Men at Arms,Soul Music,Feet of Clay,Jingo,The Fifth Elephant,The Truth,The Last Hero,Night Watch,Thud!,NaN,NaN,NaN
2,Mrs. Marietta Cosmopilite,Moving Pictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Detritus,Guards! Guards!,Moving Pictures,Men at Arms,Feet of Clay,Jingo,The Fifth Elephant,Thud!,NaN,NaN,NaN,NaN,NaN,NaN
4,Cut-Me-Own-Throat Dibbler,Guards! Guards!,Moving Pictures,Reaper Man,Soul Music,The Truth,Night Watch,Making Money,NaN,NaN,NaN,NaN,NaN,NaN
5,Dorfl,Feet of Clay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Rufus Drumknott,Feet of Clay,Jingo,The Fifth Elephant,The Truth,Night Watch,Going Postal,Making Money,Thud!,Unseen Academicals,Snuff,Raising Steam,NaN,NaN
7,Gaspode,Moving Pictures,Men at Arms,The Fifth Elephant,The Truth,Discworld Noir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Carrot,Guards! Guards!,Men at Arms,Feet of Clay,Jingo,The Fifth Elephant,The Truth,The Last Hero,Night Watch,Thud!,Snuff,NaN,NaN,NaN
9,Leonard of Quirm,Wyrd Sisters,Men at Arms,Jingo,The Fifth Elephant,The Last Hero,Discworld Noir,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
(CharBookDF_ext == 'Men at Arms').sum(axis = 1).value_counts()

1    12
0     9
dtype: int64

12 characters appeared in Men at Arms

In [20]:
CharBookDF_ext_ind = CharBookDF_ext.set_index('Character')
CharBookDF_ext_ind.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
Character,,,,,,,,,,,,,
Mrs Cake,Reaper Man,Men at Arms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fred Colon,Guards! Guards!,Men at Arms,Soul Music,Feet of Clay,Jingo,The Fifth Elephant,The Truth,The Last Hero,Night Watch,Thud!,NaN,NaN,NaN
Mrs. Marietta Cosmopilite,Moving Pictures,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Detritus,Guards! Guards!,Moving Pictures,Men at Arms,Feet of Clay,Jingo,The Fifth Elephant,Thud!,NaN,NaN,NaN,NaN,NaN,NaN
Cut-Me-Own-Throat Dibbler,Guards! Guards!,Moving Pictures,Reaper Man,Soul Music,The Truth,Night Watch,Making Money,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
CharBookDF['Unlisted'] = [(",".join(i)) for i in CharBookDF.Books]
CharBookDF.head()

,Character,Books,Unlisted
0,Mrs Cake,"[Reaper Man, Men at Arms]","Reaper Man,Men at Arms"
1,Fred Colon,"[Guards! Guards!, Men at Arms, Soul Music, Fee...","Guards! Guards!,Men at Arms,Soul Music,Feet of..."
2,Mrs. Marietta Cosmopilite,[Moving Pictures],Moving Pictures
3,Detritus,"[Guards! Guards!, Moving Pictures, Men at Arms...","Guards! Guards!,Moving Pictures,Men at Arms,Fe..."
4,Cut-Me-Own-Throat Dibbler,"[Guards! Guards!, Moving Pictures, Reaper Man,...","Guards! Guards!,Moving Pictures,Reaper Man,Sou..."


In [32]:
CharBookDF_c = CharBookDF.drop('Unlisted', axis = 1)
CharBookDF_c.head()
CharBookDF_final = CharBookDF_c.set_index('Character').Books.apply(pd.Series).stack().reset_index(level=0).rename(columns={0:'Books'})

In [51]:
MAA = CharBookDF_final.groupby('Books').count().index == 'Men at Arms'

In [52]:
CharBookDF_final.groupby('Books').count()[MAA]

,Character
Books,
Men at Arms,12


In [63]:
MC = CharBookDF_final['Character'] == 'Mrs Cake'
MC_Books = CharBookDF_final.Books[MC]
MC_Books

0     Reaper Man
1    Men at Arms
Name: Books, dtype: object

In [101]:
CV = CharBookDF_final['Character'] == 'Constable Visit'
CV_Books = CharBookDF_final.Books[CV]
CV_Books[2]

'Hogfather'

In [112]:
both_count = 0
for i in range(0,CV_Books.size):
    for j in range(0, MC_Books.size):
        if CV_Books[i] == MC_Books[j]:
            both_count = both_count + 1 
        else: both_count = both_count + 0 
both_count

0

Constable Visit and Mrs Cake were not in any book together. 